In [7]:
import pandas as pd
import gymnasium as gym
import os
from ale_py import ALEInterface, roms

frogger = gym.make("ALE/Frogger-v5", render_mode='human')

frogger.render(render_mode = "human")

RuntimeError: Failed to initialize SDL